In [6]:
from pathlib import Path
import random
import numpy as np
from PIL import Image
!pip -q install git+https://github.com/ultralytics/CLIP.git ftfy

from google.colab import drive
from huggingface_hub import login, hf_hub_download

# Install ultralytics
!pip install ultralytics

from ultralytics.models.sam import SAM3SemanticPredictor



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
drive.mount("/content/drive")



SEED = 42
PROMPTS = ["road"]
EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

random.seed(SEED)
np.random.seed(SEED)


ROOT = Path("/content/drive/MyDrive/Autonomous_Project")

IMAGES_DIR = ROOT / "data" / "subset_100" / "images"
MASKS_DIR  = ROOT / "data" / "subset_100" / "masks_sam3_road"
MASKS_DIR.mkdir(parents=True, exist_ok=True)

assert IMAGES_DIR.exists(), f"Images folder not found: {IMAGES_DIR}"



#  HF login (paste token)
login()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Download SAM3 weights

sam3_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
)
print("SAM3 weights:", sam3_path)



#Load SAM3 predictor

sam = SAM3SemanticPredictor(
    overrides=dict(
        model=sam3_path,
        task="segment",
        mode="predict",
        verbose=False,
    )
)dapsmap


SAM3 weights: /root/.cache/huggingface/hub/models--facebook--sam3/snapshots/3c879f39826c281e95690f02c7821c4de09afae7/sam3.pt


In [ ]:


#pick largest mask

def pick_largest_mask(masks_bool: np.ndarray) -> np.ndarray:
    areas = masks_bool.reshape(masks_bool.shape[0], -1).sum(axis=1)
    return masks_bool[int(areas.argmax())]





# Loop over images -> save masks
"""
Purpose:
Iterate over all images in IMAGES_DIR, generate a segmentation mask using SAM3, and save the result as a binary PNG image.

Flow:
- Collect all image files with allowed extensions.
- For each image:
  - Skip if the mask already exists.
  - Run SAM3 with the given text prompt.
  - Skip images with no returned masks.
  - Convert masks to a boolean NumPy array (N, H, W).
  - Select the largest mask.
  - Convert it to a 0/255 binary image and save it.
- Print progress and finish when done.
"""


img_paths = [p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in EXTS]
print(f"Found {len(img_paths)} images")

for img_path in img_paths:
    out_path = MASKS_DIR / f"{img_path.stem}_mask.png"
    if out_path.exists():
        continue

    sam.set_image(str(img_path))
    results = sam(text=PROMPTS)

    if not results or results[0].masks is None:
        print(f"[SKIP] no masks: {img_path.name}")
        continue

    masks = results[0].masks.data.detach().cpu().numpy().astype(bool)  # (N,H,W)
    if masks.shape[0] == 0:
        print(f"[SKIP] empty masks: {img_path.name}")
        continue

    best_mask = pick_largest_mask(masks)
    mask_255 = best_mask.astype(np.uint8) * 255

    Image.fromarray(mask_255).save(out_path)
    print(f"[OK] {img_path.name} -> {out_path.name}")

print("Done.")

Found 100 images
Ultralytics 8.3.239 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
Results saved to /content/runs/segment/predict2
[SKIP] no masks: 000430.png
WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
Results saved to /content/runs/segment/predict2
[SKIP] no masks: 000465.png
WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
Results saved to /content/runs/segment/predict2
[SKIP] no masks: 000216.png
WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
Results saved to /content/runs/segment/predict2
[SKIP] no masks: 000730.png
WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
Results saved to /content/runs/segment/predict2
[SKIP] no masks: 000381.png
WARNING ⚠️ imgsz=[640] must be multiple of max stride 14, updating to [644]
Results saved to /content/runs/segment/predict2
[SKIP] no masks

In [12]:
from pathlib import Path
MASKS_DIR = Path("/content/drive/MyDrive/Autonomous_Project/data/subset_100/masks_sam3_road")
print("done:", len(list(MASKS_DIR.glob("*_mask.png"))))


done: 91
